# Installing Necessary Libraries


In [ ]:
!pip install faiss-gpu
!pip install deepface

     |████████████████████████████████| 35.5MB 88kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.11.1-cp36-none-any.whl size=9656 sha256=8e62b5c3c88f7aed093606f093bd7cf51f7815acde17b955c821d2912f909bb1
  Stored in directory: /root/.cache/pip/wheels/32/8e/8d/0fec7d933759a0880dc34be487e1d92c51ba8fdf4e9f265ff7
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4


# Importing Libraries

In [12]:
from deepface import DeepFace
from deepface.basemodels import Facenet,FbDeepFace,VGGFace,OpenFace
# from deepface.basemodels import DeepFace
from deepface.commons import functions, realtime, distance as dst
import pandas as pd
import numpy as np
import faiss
from faiss import *
import os
import timeit
import time

# Configuration for model and Similarity Metric

In [64]:
model = Facenet # Or FbDeepFace Or VGGFace Or OpenFace
if(model==Facenet):
  model = Facenet.loadModel()
  dim = 128
  ip_shape = 160
if(model==FbDeepFace):
  model = FbDeepFace.loadModel()
  dim = 4096
  ip_shape = 152
if(model==VGGFace):
  model = VGGFace.loadModel()
  dim = 2622
  ip_shape = 224
if(model==OpenFace):
  model = OpenFace.loadModel()
  dim = 128
  ip_shape = 96

Similarity_Metric = 1 # 0 for L2 distance Similarity and 1 for Cosine Similarity
# source - https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

# Inference

In [61]:
def get_face_encodings(file_name,model):
    img1 = functions.detectFace(file_name,(ip_shape, ip_shape))
    start_time = time.time()
    img1_representation = model.predict(img1)[0,:]
    print("--- %s seconds ---" % (time.time() - start_time))
    # print("Done")
    return img1_representation

def register_faces_in_dataset(faces_dataset,model):
  images=os.listdir(faces_dataset)
  f = open("faces.csv","w")
  first_row = str()
  first_row = first_row + "id"
  for i in range(1,dim+1):
    first_row = first_row + "," + str(i)
  first_row = first_row + "\n"
  f.write(first_row)
  for image in images:
    # print(image)
    encoding = get_face_encodings(faces_dataset+"//"+image,model)
    string = str()

    name = image.split(".")
    name = name[0]
    string = string + name
    for i in encoding:
      string = string + "," + str(i)
    string = string+"\n"
    # print(string)
    f.write(string)
  f.close()

def getting_id_index(faces_file):     # for getting faces identities and corresponding encodings in numpy format for matching.
  df = pd.read_csv(faces_file)
  face_identities = []
  if(Similarity_Metric==0):
    face_index = faiss.IndexFlatL2(dim)
  if(Similarity_Metric==1):
    face_index = faiss.IndexFlatIP(dim)
  print(face_index.is_trained)

  for _, row in df.iterrows():
      identity = row["id"]
      # print(row.iloc[1:dim+1])
      embedding = row.iloc[1:dim+1].to_numpy().astype('float32')
      embedding = np.ascontiguousarray(embedding.reshape(1, dim))
      # print(embedding)
      # embedding = embedding[0]
      if(Similarity_Metric==1):
        normalize_L2(embedding)
      # print(embedding)

      face_index.add(embedding)
      face_identities.append(identity)
  print(face_identities)
  return face_identities, face_index

def find_face_name(image_name,model,face_index,face_identities): # for matching
  encoding = get_face_encodings(image_name,model)
  # print(encoding)
  encoding = np.reshape(encoding, (1, dim))
  if(Similarity_Metric==1):
    normalize_L2(encoding)
  distances, indicies = face_index.search(encoding, 1)
  print(distances)
  return(face_identities[indicies[0][0]])


In [62]:
register_faces_in_dataset("Faces-Dataset",model)
face_identities, face_index = getting_id_index("faces.csv")

--- 4.411850214004517 seconds ---
--- 0.03307819366455078 seconds ---
--- 0.033742427825927734 seconds ---
--- 0.0340878963470459 seconds ---
True
['Hrithik', 'Robert-Downey-Jr', 'Shahrukh-khan', 'Chris']


In [63]:
face_name = find_face_name("face3.jpg",model,face_index,face_identities) 
print(face_name)

--- 0.037253379821777344 seconds ---
[[0.8118255]]
Robert-Downey-Jr
